In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-cufft-10-1 cuda-cufft-dev-10-1 cuda-curand-10-1 cuda-curand-dev-10-1
  cuda-cusolver-10-1 cuda-cusolver-dev-10-1 cuda-cusparse-10-1
  cuda-cusparse-dev-10-1 cuda-drivers cuda-license-10-2 cuda-npp-10-1
  cuda-npp-dev-10-1 cuda-nsight-10-1 cuda-nsight-compute-10-1
  cuda-nsight-systems-10-1 cuda-nvgraph-10-1 cuda-nvgraph-dev-10-1
  cuda-nvjpeg-10-1 cuda-nvjpeg-dev-10-1 cuda-nvrtc-10-1 cuda-nvrtc-dev-10-1
  cuda-nvvp-10-1 default-jre dkms freeglut3 freeglut3-dev
  keyboard-configuration libargon2-0 libcap2 libcryptsetup12 libcublas10
  libdevmapper1.02.1 libfontenc1 libgtk2.0-0 libgtk2.0-common libidn11
  libip4tc0 libjansson4 libnvidia-cfg1-440 libnvidia-common-430
  libnvidia-common-440 libnvidia-decode-440 libnvidia-encode-440
  libnvidia-fbc1-440 libnvidia-gl-440 libnvidia-ifr1-440 libpam-systemd
  libpo

In [0]:
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [0]:
image_path_in_colab="/content/drive/My Drive/Colab Notebooks/edo testing/15.jpg"

In [0]:
extractedInformation = pytesseract.image_to_string(Image.open(image_path_in_colab))
print(extractedInformation)

Oh...
Okay...


# **Tesseract OCR Text Recognition**

In [0]:
image_path_in_colab="/content/drive/My Drive/Colab Notebooks/edo testing/7 (4).jpg"

In [0]:
# import the necessary packages
from imutils.object_detection import non_max_suppression
import numpy as np
import pytesseract
import argparse
import cv2
from google.colab.patches import cv2_imshow

## Text Detection

In [0]:
min_confidence = 0.95

def decode_predictions(scores, geometry):
	# grab the number of rows and columns from the scores volume, then
	# initialize our set of bounding box rectangles and corresponding
	# confidence scores
	(numRows, numCols) = scores.shape[2:4]
	rects = []
	confidences = []
 
	# loop over the number of rows
	for y in range(0, numRows):
		# extract the scores (probabilities), followed by the
		# geometrical data used to derive potential bounding box
		# coordinates that surround text
		scoresData = scores[0, 0, y]
		xData0 = geometry[0, 0, y]
		xData1 = geometry[0, 1, y]
		xData2 = geometry[0, 2, y]
		xData3 = geometry[0, 3, y]
		anglesData = geometry[0, 4, y]
 
		# loop over the number of columns
		for x in range(0, numCols):
			# if our score does not have sufficient probability,
			# ignore it
			if scoresData[x] < min_confidence:
				continue
 
			# compute the offset factor as our resulting feature
			# maps will be 4x smaller than the input image
			(offsetX, offsetY) = (x * 4.0, y * 4.0)
 
			# extract the rotation angle for the prediction and
			# then compute the sin and cosine
			angle = anglesData[x]
			cos = np.cos(angle)
			sin = np.sin(angle)
 
			# use the geometry volume to derive the width and height
			# of the bounding box
			h = xData0[x] + xData2[x]
			w = xData1[x] + xData3[x]
 
			# compute both the starting and ending (x, y)-coordinates
			# for the text prediction bounding box
			endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
			endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
			startX = int(endX - w)
			startY = int(endY - h)
 
			# add the bounding box coordinates and probability score
			# to our respective lists
			rects.append((startX, startY, endX, endY))
			confidences.append(scoresData[x])
 
	# return a tuple of the bounding boxes and associated confidences
	return (rects, confidences)

In [0]:
width = 2976
height = 3968
#width = 384
#height = 512

# load the input image and grab the image dimensions
image = cv2.imread(image_path_in_colab)
orig = image.copy()
(origH, origW) = image.shape[:2]

# set the new width and height and then determine the ratio in change
# for both the width and height
if origH >= origW:
    newH = height
    newW = width  # round(new_h*W/H)
else:
    newW = height
    newH = width  # round(new_w*H/W)
rW = origW / float(newW)
rH = origH / float(newH)
 
# resize the image and grab the new image dimensions
image = cv2.resize(image, (newW, newH))
(H, W) = image.shape[:2]


In [0]:
image.shape

(3968, 2976, 3)

In [0]:
# define the two output layer names for the EAST detector model that
# we are interested -- the first is the output probabilities and the
# second can be used to derive the bounding box coordinates of text
layerNames = [
    "feature_fusion/Conv_7/Sigmoid",
    "feature_fusion/concat_3"]

In [0]:
# load the pre-trained EAST text detector
print("[INFO] loading EAST text detector...")
net = cv2.dnn.readNet('/content/drive/My Drive/Colab Notebooks/edo testing/frozen_east_text_detection.pb')

[INFO] loading EAST text detector...


In [0]:
# construct a blob from the image and then perform a forward pass of
# the model to obtain the two output layer sets
blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
    (123.68, 116.78, 103.94), swapRB=True, crop=False)
net.setInput(blob)
(scores, geometry) = net.forward(layerNames)

In [0]:
# decode the predictions, then  apply non-maxima suppression to
# suppress weak, overlapping bounding boxes
(rects, confidences) = decode_predictions(scores, geometry)
boxes = non_max_suppression(np.array(rects), probs=confidences)

In [0]:
confidences

[0.76287794,
 0.99860233,
 0.9986871,
 0.8160829,
 0.874692,
 0.9815825,
 0.91031104,
 0.97160745,
 0.9940268,
 0.9863144,
 0.842608,
 0.96983254,
 0.94940764,
 0.995458,
 0.9750455,
 0.96439064,
 0.9968845,
 0.99849665,
 0.9452935,
 0.9492905,
 0.8182547,
 0.71858454,
 0.9898217,
 0.999532,
 0.99668247,
 0.9997336,
 0.999652,
 0.9991773,
 0.9994879,
 0.86341417,
 0.82378083,
 0.9978288,
 0.99919087,
 0.99697685,
 0.9957339,
 0.9979929,
 0.9978332,
 0.9979924,
 0.987298,
 0.9308915,
 0.86386704,
 0.99577624,
 0.85211277,
 0.9046641,
 0.7069167,
 0.9985196,
 0.9974578,
 0.9981198,
 0.99746275,
 0.98530406,
 0.98549366,
 0.90395457,
 0.99874085,
 0.9834812,
 0.986797,
 0.8950035,
 0.78460836,
 0.8284143,
 0.7864186,
 0.98582524,
 0.91953593,
 0.9436172,
 0.9918139,
 0.962513,
 0.8604001,
 0.7116157,
 0.78925085,
 0.94402754,
 0.99913496,
 0.9992155,
 0.9991518,
 0.9864238,
 0.9352778,
 0.9821768,
 0.82238734,
 0.985399,
 0.998892,
 0.99779606,
 0.97078395,
 0.99904364,
 0.999408,
 0.9997

## Text Recognition

In [0]:
padding = 0.2

# initialize the list of results
results = []
 
# loop over the bounding boxes
for (startX, startY, endX, endY) in boxes:
	# scale the bounding box coordinates based on the respective
	# ratios
	startX = int(startX * rW)
	startY = int(startY * rH)
	endX = int(endX * rW)
	endY = int(endY * rH)
 
	# in order to obtain a better OCR of the text we can potentially
	# apply a bit of padding surrounding the bounding box -- here we
	# are computing the deltas in both the x and y directions
	dX = int((endX - startX) * padding)
	dY = int((endY - startY) * padding)
 
	# apply padding to each side of the bounding box, respectively
	startX = max(0, startX - dX)
	startY = max(0, startY - dY)
	endX = min(origW, endX + (dX * 2))
	endY = min(origH, endY + (dY * 2))
 
	# extract the actual padded ROI
	roi = orig[startY:endY, startX:endX]

  # in order to apply Tesseract v4 to OCR text we must supply
	# (1) a language, (2) an OEM flag of 4, indicating that the we
	# wish to use the LSTM neural net model for OCR, and finally
	# (3) an OEM value, in this case, 7 which implies that we are
	# treating the ROI as a single line of text
	config = ("-l eng --oem 1 --psm 7")
	text = pytesseract.image_to_string(roi, config=config)
 
	# add the bounding box coordinates and OCR'd text to the list
	# of results
	results.append(((startX, startY, endX, endY), text))

In [0]:
# sort the results bounding box coordinates from top to bottom
results = sorted(results, key=lambda r:r[0][1])
 
# loop over the results
for ((startX, startY, endX, endY), text) in results:
	# display the text OCR'd by Tesseract
	print("OCR TEXT")
	print("========")
	print("{}\n".format(text))
 
	# strip out non-ASCII text so we can draw the text on the image
	# using OpenCV, then draw the text and a bounding box surrounding
	# the text region of the input image
	text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
	output = orig.copy()
	cv2.rectangle(output, (startX, startY), (endX, endY),
		(0, 0, 255), 2)
	cv2.putText(output, text, (startX, startY - 20),
		cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
 
	# show the output image
	#cv2_imshow(output)
	cv2.waitKey(0)

OCR TEXT
GIBUNG

OCR TEXT
YWSUIP 1

OCR TEXT
1? TANIGORO | 9

OCR TEXT
AH FARMASII Tigges

OCR TEXT
BSEKOLAH FARMA

